Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from math import exp
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix


KeyboardInterrupt



Importing datasets

In [ ]:
data1 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/sales_train.csv', encoding = 'utf-8')
data2 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/sales_test.csv', encoding = 'utf-8')
data3 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/calendar.csv', encoding = 'utf-8')
data4 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/inventory.csv', encoding = 'utf-8')
data5 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/solution.csv', encoding = 'utf-8')
data6 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/test_weights.csv', encoding = 'utf-8')
print("Datasets successfully imported")

In [ ]:
print(data1.info())
print(data1.isnull().sum()) #info of train set

In [ ]:
pd.set_option('display.max_rows', None)
null_count = data1[data1['total_orders'].isnull() & data1['sales'].isnull()]['date']
print(null_count)

In [ ]:
pd.set_option('display.max_rows', None)
matching_rows = data3[data3['date'].isin(null_count)][['date', 'holiday', 'warehouse']]
print(matching_rows)

In [ ]:
for date in null_count:
    data1.loc[data1['date'] == date, 'total_orders'] = data1.loc[
    data1['date'] <= date, 'total_orders'].rolling(window = 7, min_periods = 1).mean().iloc[-1]
    data1.loc[data1['date'] == date, 'sales'] = data1.loc[
    data1['date'] <= date, 'sales'].rolling(window = 7, min_periods = 1).mean().iloc[-1]
print(data1.isnull().sum())
print(data1.shape)

In [ ]:
print(data2.info())
print(data2.isnull().sum()) #info of test set

In [ ]:
data2.date.value_counts().sort_index()

In [ ]:
print(data3.info())
print(data3.isnull().sum()) #info of calender set

In [ ]:
print(data4.info())
print(data4.isnull().sum()) #info of inventory set

In [ ]:
print(data5.info())
print(data5.isnull().sum()) #info of solution set

In [ ]:
print(data6.info())
print(data6.isnull().sum()) #info of test weights set

In [ ]:
merge1 = pd.merge(data1, data6, on = 'unique_id', how = 'left')
print(merge1.shape)

In [ ]:
merge2 = pd.merge(merge1, data3, on = ['date', 'warehouse'], how = 'left')
print(merge2.shape)

In [ ]:
merge3 = pd.merge(merge2, data4, on = ['unique_id', 'warehouse'], how = 'left')
print(merge3.shape)

In [ ]:
merge3['date'] = pd.to_datetime(merge3['date'], errors = 'coerce')
merge3['day'] = merge3['date'].dt.day
merge3['month'] = merge3['date'].dt.month
merge3['year'] = merge3['date'].dt.year
print(merge3.info())

In [ ]:
merge4 = pd.merge(data2, data3, on = ['date', 'warehouse'], how = 'left')
print(merge4.shape)

In [ ]:
merge5 = pd.merge(merge4, data4, on = ['unique_id', 'warehouse'], how = 'left')
print(merge5.shape)

In [ ]:
merge5['date'] = pd.to_datetime(merge5['date'], errors = 'coerce')
merge5['day'] = merge5['date'].dt.day
merge5['month'] = merge5['date'].dt.month
merge5['year'] = merge5['date'].dt.year
print(merge5.info())

In [ ]:
merge3['log_sales'] = np.log(merge3['sales'] + 1)
merge3.info()

In [ ]:
#taken from https://www.kaggle.com/code/yzokulu/rohlik-sales-forecasting-xgbregressor
from datetime import datetime
czech_holiday = [ 
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"),
]

brno_holiday = [
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"),
]

budapest_holidays = []

munich_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),
]

frankfurt_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),
]

def fill_loss_holidays(fill, warehouses, holidays):
    merge3 = fill.copy()
    for item in holidays:
        dates, holiday_name = item
        generated_dates = [datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d') for date in dates]
        for generated_date in generated_dates:
            merge3.loc[(merge3['warehouse'].isin(warehouses)) & (merge3['date'] == generated_date), 'holiday'] = 1
            merge3.loc[(merge3['warehouse'].isin(warehouses)) & (merge3['date'] == generated_date), 'holiday_name'] = holiday_name
    return merge3

merge3 = fill_loss_holidays(fill=merge3, warehouses=['Prague_1', 'Prague_2', 'Prague_3'], holidays=czech_holiday)
merge3 = fill_loss_holidays(fill=merge3, warehouses=['Brno_1'], holidays=brno_holiday)
merge3 = fill_loss_holidays(fill=merge3, warehouses=['Munich_1'], holidays=munich_holidays)
merge3 = fill_loss_holidays(fill=merge3, warehouses=['Frankfurt_1'], holidays=frankfurt_holidays)
merge3 = fill_loss_holidays(fill=merge3, warehouses=['Budapest_1'], holidays=budapest_holidays)
print("Holidays added Successfully")

In [ ]:
merge3.drop(columns=['availability'], inplace=True)
print("Column Dropped Successfully")

In [ ]:
#part of the code is taken from https://www.kaggle.com/code/yzokulu/rohlik-sales-forecasting-xgbregressor

merge3['is_test'] = 0
merge5['is_test'] = 1

merged_data = pd.concat([merge3, merge5], axis=0)


discount = ['type_0_discount', 'type_1_discount', 'type_2_discount', 
                    'type_3_discount', 'type_4_discount', 'type_5_discount', 'type_6_discount']
merged_data['max_discount'] = merged_data[discount].max(axis=1)

merged_data = merged_data.sort_values(by=['product_unique_id', 'warehouse', 'date'])

periods = [3,7,10,14]
for p in periods: 
    
    merged_data['total_orders_mean_' + str(p)] = merged_data.groupby(['product_unique_id', 'warehouse'])['total_orders'] \
    .rolling(window=p, min_periods=1) \
    .mean() \
    .reset_index(level=[0, 1], drop=True)

    merged_data['total_orders_std_' + str(p)] = merged_data.groupby(['product_unique_id', 'warehouse'])['total_orders'] \
    .rolling(window=p, min_periods=1) \
    .std() \
    .reset_index(level=[0, 1], drop=True)

    merged_data['total_orders_max_' + str(p)] = merged_data.groupby(['product_unique_id', 'warehouse'])['total_orders'] \
    .rolling(window=p, min_periods=1) \
    .max() \
    .reset_index(level=[0, 1], drop=True)

    merged_data['total_orders_min_' + str(p)] = merged_data.groupby(['product_unique_id', 'warehouse'])['total_orders'] \
    .rolling(window=p, min_periods=1) \
    .min() \
    .reset_index(level=[0, 1], drop=True)

for lag in range(1, 15):  
    merged_data[f'lag_{lag}'] = merged_data.groupby(['product_unique_id', 'warehouse'])['sales'].shift(lag)
    merged_data[f'lag_{lag}'] = merged_data[f'lag_{lag}'].fillna(merged_data.groupby(['product_unique_id', 'warehouse'])['sales'].transform("last"))

In [ ]:
categorical_columns = ['warehouse', 'name', 'L1_category_name_en', 'L2_category_name_en', 
                       'L3_category_name_en', 'L4_category_name_en', 'holiday_name']
   
for col in categorical_columns:
    merged_data[col] = merged_data[col].astype('category')
    merged_data[col] = merged_data[col].cat.codes

In [ ]:
merged_data["datetime"] = pd.to_datetime(merged_data["date"])
merged_data["month"] = merged_data["datetime"].dt.month
merged_data["day"] = merged_data["datetime"].dt.day
merged_data["weekday"] = merged_data["datetime"].dt.weekday
merged_data["quarter"] = merged_data["datetime"].dt.quarter
merged_data["week_of_year"] = merged_data["datetime"].dt.isocalendar().week
merged_data["day_of_year"] = merged_data["datetime"].dt.dayofyear
merged_data["is_weekend"] = merged_data["datetime"].dt.weekday.isin([5, 6]).astype(int)
merged_data["is_month_start"] = merged_data["datetime"].dt.is_month_start.astype(int)
merged_data["is_month_end"] = merged_data["datetime"].dt.is_month_end.astype(int)
merged_data['year_sin'] = np.sin(2 * np.pi * merged_data['datetime'].dt.year)
merged_data['year_cos'] = np.cos(2 * np.pi * merged_data['datetime'].dt.year)
merged_data['month_sin'] = np.sin(2 * np.pi * merged_data['month'] / 12) 
merged_data['month_cos'] = np.cos(2 * np.pi * merged_data['month'] / 12)
merged_data['day_sin'] = np.sin(2 * np.pi * merged_data['day'] / 31)  
merged_data['day_cos'] = np.cos(2 * np.pi * merged_data['day'] / 31)
    
merged_data.drop(columns=["datetime"], inplace=True)

merged_data = merged_data.astype({'week_of_year': 'int32'})

In [ ]:
merge3_new = merged_data[(merged_data['is_test'] == 0) & (merged_data['date'] < '2024-05-20') & (merged_data['date'] >= '2023-01-01')]
sales_val = merged_data[(merged_data['is_test'] == 0) & (merged_data['date'] >= '2024-05-21') & (merged_data['date'] <= '2024-06-02')]

merge3_new = merge3_new.dropna()
sales_val = sales_val.dropna()

merge5_new = merged_data[merged_data['is_test'] == 1]

In [ ]:
features = [
    "warehouse", "total_orders", "sell_price_main", "max_discount", 
    "name", "holiday", "holiday_name", 
    "shops_closed", "winter_school_holidays", "school_holidays", "total_orders_mean_3", "total_orders_std_3", "total_orders_max_3", 
    "total_orders_min_3", "total_orders_mean_7", "total_orders_std_7", "total_orders_max_7", "total_orders_min_7", "total_orders_mean_10", "total_orders_std_10", "total_orders_max_10", 
    "total_orders_min_10", "total_orders_mean_14", "total_orders_std_14", "total_orders_max_14", "total_orders_min_14", "month", "day", "weekday", "quarter", "week_of_year", 
    "day_of_year", "is_weekend", "is_month_start", "is_month_end",
    "lag_1", "lag_2", "lag_3", "lag_4", "lag_5", "lag_6", "lag_7", "lag_8", "lag_9", "lag_10", "lag_11", "lag_12", "lag_13", "lag_14"] 

X_train = merge3_new[features]
y_train = merge3_new['log_sales']
X_val = sales_val[features]
y_val = sales_val['log_sales']
X_test = merge5_new[features]

model = CatBoostRegressor(
    iterations=10_000,
    #learning_rate=0.1,
    depth=10,
    loss_function="MAE",
    verbose=100,
    random_seed=0)

model = LGBMRegressor(
    boosting_type='gbdt',  
    objective='regression',  
    metric='mae',  
    num_leaves=31,  
    learning_rate=0.1,  
    feature_fraction=0.9,  
    bagging_fraction=0.8,  
    bagging_freq=5,  
    verbose=-1,  
    n_jobs=-1,  
    random_state=42,  
    n_estimators=200)

params = {
    'eval_metric': 'mae', 
    'learning_rate': 0.1,  
    'max_depth': 11,  
    'min_child_weight': 1,  
    'subsample': 0.8,  
    'colsample_bytree': 0.8,  
    'n_estimators': 200,  
    'n_jobs': -1,  
    'seed': 42}

model = XGBRegressor(**params)

model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=100)

y_val_pred = model.predict(X_val)
y_test_pred = np.exp(model.predict(X_test)) - 1

mae = mean_absolute_error(y_val, y_val_pred)
print(f'Mean Absolute Error on validation set: {mae}')
merge5_new['predicted_sales'] = y_test_pred

In [ ]:
# XGBoost Model evaluation (using MAE)
y_train_pred_xgb = model.predict(X_train)
y_val_pred_xgb = model.predict(X_val)  # Predict for the validation set

# Calculate MAE on training and validation sets
mae_train_xgb = mean_absolute_error(y_train, y_train_pred_xgb)
mae_val_xgb = mean_absolute_error(y_val, y_val_pred_xgb)

print(f'Mean Absolute Error on training set (XGBoost): {mae_train_xgb}')
print(f'Mean Absolute Error on validation set (XGBoost): {mae_val_xgb}')

# LightGBM Model evaluation (using MAE)
y_train_pred_lgbm = model.predict(X_train)
y_val_pred_lgbm = model.predict(X_val)

# Calculate MAE on training and validation sets for LightGBM
mae_train_lgbm = mean_absolute_error(y_train, y_train_pred_lgbm)
mae_val_lgbm = mean_absolute_error(y_val, y_val_pred_lgbm)

print(f'Mean Absolute Error on training set (LightGBM): {mae_train_lgbm}')
print(f'Mean Absolute Error on validation set (LightGBM): {mae_val_lgbm}')

In [ ]:
merge5_new['id'] = merge5_new['unique_id'].astype(str) + '_' + merge5_new['date'].astype(str) 
results = merge5_new[['id', 'predicted_sales']].copy() 
results.columns = ['id', 'sales_hat'] 
results.to_csv('submission.csv', index=False)

In [ ]:
min_length = min(len(y_val_pred_xgb), len(y_test_pred))
mae = mean_absolute_error(y_val_pred_xgb[:min_length], y_test_pred[:min_length])
mse = MSE(y_val_pred_xgb[:min_length], y_test_pred[:min_length])
rmse = np.sqrt(mse)
r2 = r2_score(y_val_pred_xgb[:min_length], y_test_pred[:min_length])

print(f"R²: {r2}")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

In [ ]:
threshold = 0.5
min_length = min(len(y_val_pred_xgb), len(y_test_pred))
y_pred_class = np.where(y_test_pred[:min_length] >= threshold, 1, 0)
val_y_class = np.where(y_val_pred_xgb[:min_length] >= threshold, 1, 0)

accuracy = accuracy_score(val_y_class, y_pred_class)
f1 = f1_score(val_y_class, y_pred_class, average='weighted')
precision = precision_score(val_y_class, y_pred_class, average='weighted')
recall = recall_score(val_y_class, y_pred_class, average='weighted')
cm = confusion_matrix(val_y_class, y_pred_class)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Confusion Matrix:\n{cm}")